In [6]:
import numpy as np
import pandas as pd
import datetime as dt
import os
import scipy.optimize as opt
from scipy.optimize import LinearConstraint, Bounds

In [8]:
#reading in orlin's cleaned csv data
local_filepath = "/Users/leonlu/Downloads/gold_data.csv"
gold_data = pd.read_csv(local_filepath, index_col=0)
display(gold_data.head())

,date,AUSHF,dateNoH,ATM_C,OTM_C,ITM_C,ATM_P,OTM_P,ITM_P
0,2020-05-07 01:00:00,378.00,2020-05-07,7.48,5.70,10.10,5.98,4.28,8.24
1,2020-05-08 01:00:00,381.86,2020-05-08,6.90,5.16,8.96,5.32,3.68,7.66
2,2020-05-09 01:00:00,381.68,2020-05-09,6.76,4.80,9.22,5.26,3.56,7.06
3,2020-05-12 01:00:00,380.32,2020-05-12,5.20,3.74,7.66,5.26,3.58,7.86
4,2020-05-13 01:00:00,380.70,2020-05-13,5.10,3.50,7.78,4.22,2.82,6.96


In [12]:
#construct reward matrix R_{i,t} of dimension (K x N)
reward_df = gold_data.iloc[:, -6:].pct_change()
reward_df.dropna(how = 'all', inplace = True)
reward_df.drop(reward_df.index[-1], inplace=True) #drops the last line of data
display(reward_df)

,ATM_C,OTM_C,ITM_C,ATM_P,OTM_P,ITM_P
1,-0.077540,-0.094737,-0.112871,-0.110368,-0.140187,-0.070388
2,-0.020290,-0.069767,0.029018,-0.011278,-0.032609,-0.078329
3,-0.230769,-0.220833,-0.169197,0.000000,0.005618,0.113314
4,-0.019231,-0.064171,0.015666,-0.197719,-0.212291,-0.114504
5,0.313725,0.297143,0.210797,-0.180095,-0.212766,-0.235632
...,...,...,...,...,...,...
507,0.167109,0.153846,0.160896,-0.096463,-0.092827,-0.083951
508,-0.127273,-0.126984,-0.122807,0.149466,0.144186,0.118598
509,-0.059896,-0.050909,-0.016000,-0.241486,-0.292683,-0.166265
510,0.177285,0.168582,0.170732,0.024490,0.097701,-0.052023


In [14]:
K = 6 #number of peripheral assets (non AUSHF) in market
gamma_ = 0.95 #confidence level for CVaR performance function F
lambda_ = np.random.uniform(0, 1) #risk preference (->1 means UCB1, ->0 means min CVaR)
N = 10 #number of time-steps/episodes

In [15]:
#define helper functions used in huo/fu paper
def I(t):
    """
    Definition: Equation 2.1; follows the UCB1 algorithm.
    t: the time
    """
    def R_bar(i, t):
        subset = reward_df.iloc[:t+1, i]
        return subset.mean()
    if t <= K:
        return t
    else:
        R_bars = []
        for j in range(len(reward_df.columns)):
            R_bars.append(R_bar(j, t))
        return np.argmax(R_bars) + np.sqrt(2 * np.log(t) / T(i, t-1)) #need to define T
        
def F(gamma, u, alpha, t, delta):
    """
    Definition: Equation 2.3; estimates the conditional value-at-risk.
    H_matrix: the s-th column of H_matrix (K rows, \delta columns) is the historical returns of the s-th asset
    R_matrix: the s-th column of R_matrix (K rows, t - 1 columns) is the trial of returns observed so far of the s-th asset
    """
    sum1 = np.sum([np.max(-np.dot(historical_df.iloc[:, s], u) - alpha, 0) for s in range(delta)])
    sum2 = np.sum([np.max(-np.dot(reward_df.iloc[:, s], u) - alpha, 0) for s in range(t - 1)])
    return alpha + (sum1 + sum2) / ((delta + t - 1) * (1 - gamma))

In [ ]:
#computation of different omega components
def omega_C(t):
    """
    Definition: Equation 2.4
    Computes a risk-aware portfolio according to Equation 2.3.
    t: the time
    """
    # the objective function at iteration t
    def F_t(args):
        u, alpha = args[0], args[1]
        return F(gamma, u, alpha, t, H_matrix, R_matrix)

    # constraining u to be a probability distribution over 1, 2, ..., K
    sum_to_one = LinearConstraint(np.ones(K), [0], [1])
    positivity_bounds = Bounds(np.zeros(K), np.ones(K))

    # scipy.optimize.minimize
    x0 = np.ones(K + 1)
    x0[:K] /= K # first K components should sum to 1; no idea what alpha (last component) should represent
    approx_min = opt.minimize(
        F_t, x0=x0,
        constraints=[sum_to_one],
        bounds=positivity_bounds
    )
    return approx_min
def omega_M(t):
    e = np.zeros(K)
    one_i = I(t)
    e[one_i] = 1
    return e

In [ ]:
def sequential_selection_algo(K, lambda_, gamma=gamma, time_horizon=time_horizon):
    """
    K: number of assets
    gamma: confidence level
    lambda_: risk preference
    """
    returns = []
    for t in range(time_horizon):
        # Equation. 2.2: compute omega_M(t)
        omega_M = np.zeros(K)
        omega_M[I(t)] = 1
        
        # Equation 2.4: compute the risk aware portfolio
        risk_aware_portfolio = omega_C(t)

        # Equation 2.5: compute convex combination of UCB1 portfolio and risk aware portfolio
        omega_star = lambda_ * omega_M + (1 - lambda_) * risk_aware_portfolio

        # "Receive portfolio reward"
        returns_by_t = np.dot(omega_star, R_matrix[:, t])
        returns.append(returns_by_t)
    return returns # idk what to return; there might be a formula somewhere